In [1]:
import torch

from transformers import (AutoTokenizer,
                          AutoModelForSeq2SeqLM,
                          Seq2SeqTrainingArguments,
                          Seq2SeqTrainer,
                          DataCollatorForSeq2Seq)

from datasets import Dataset
from sklearn.metrics import classification_report
from tqdm import tqdm

%load_ext autoreload
%autoreload 2
from roberta_type_models_testing import load_data

In [2]:
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def generate_sample(text):
  return f"""Perform Sentiment classification task.
Given the text assign a sentiment label from ['negative', 'positive', 'neutral'].
Return label only without any other text.

<text>: {text}
<sentiment>: """.strip()

def run_on_test(test_dataset, model, str2int):
    golden_labels = []
    predicted_labels = []

    test_texts, test_labels = test_dataset['text'], test_dataset['label']
    inputs = [generate_sample(text) for text in test_texts]

    # Running inference on 1 sample at a time to avoid OOM issue
    for i, input in enumerate(tqdm(inputs)):
      input = tokenizer(input, return_tensors='pt').to('cuda')
      output = model.generate(**input)

      golden_labels.append(str2int[test_labels[i]])
      predicted_labels.append(str2int[tokenizer.decode(output[0], skip_special_tokens=True)])

    return golden_labels, predicted_labels

In [3]:
default_single_seed = 77
few_shot_samples = 12
stratified_seeds = [55, 66, 77, 88, 99]
int2str = {-1: 'negative', 0: 'neutral', 1: 'positive'}
str2int = {v:k for k, v in int2str.items()}

def preprocess_function(examples):
    inputs = [generate_sample(text) for text in examples['text']]
    model_inputs = tokenizer(inputs, max_length=512,  truncation=True)

    # The labels are tokenized outputs
    labels = tokenizer(text_target=examples['label'],
                        max_length=512,
                        truncation=True)

    model_inputs['labels'] = labels['input_ids']

    return model_inputs

for i in range(5):
    train_ds, test_ds = load_data("data/train_data.csv",
                                  "data/test_data.csv",
                                  few_shot_samples,
                                  tokenizer,
                                  512,
                                  model_name,
                                  stratified_seeds[i])
    
    tokenized_train_dataset = train_ds.map(preprocess_function, batched=True)
    tokenized_test_dataset = test_ds.map(preprocess_function, batched=True)
    tokenized_train_dataset = tokenized_train_dataset.remove_columns(['text', 'label'])
    tokenized_test_dataset = tokenized_test_dataset.remove_columns(['text', 'label'])
    
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

    training_args = Seq2SeqTrainingArguments(
        output_dir="./results",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        weight_decay=0.01,
        num_train_epochs=3,
        evaluation_strategy="epoch",
        logging_strategy = 'epoch',
        save_strategy="epoch",
        predict_with_generate=True,
        push_to_hub=False,
        load_best_model_at_end=True)
    
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_test_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator)
    
    trainer.train()

    golden_labels, predicted_labels = run_on_test(test_ds, model, str2int)
    print(classification_report(golden_labels, predicted_labels, digits=4))
    
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


  0%|          | 0/6 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.5218, 'learning_rate': 1.3333333333333333e-05, 'epoch': 1.0}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.28298136591911316, 'eval_runtime': 0.8405, 'eval_samples_per_second': 93.992, 'eval_steps_per_second': 11.898, 'epoch': 1.0}
{'loss': 0.4506, 'learning_rate': 6.666666666666667e-06, 'epoch': 2.0}


...

  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.28749600052833557, 'eval_runtime': 1.0498, 'eval_samples_per_second': 75.254, 'eval_steps_per_second': 9.526, 'epoch': 3.0}
{'train_runtime': 17.3709, 'train_samples_per_second': 2.072, 'train_steps_per_second': 0.345, 'train_loss': 0.4198189477125804, 'epoch': 3.0}


  0%|          | 0/79 [00:00<?, ?it/s]c:\Users\Martin\Desktop\projects\master thesis\master_thesis\Lib\site-packages\transformers\generation\utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 79/79 [00:02<00:00, 34.74it/s]


              precision    recall  f1-score   support

          -1     0.7333    0.9565    0.8302        23
           0     0.8125    0.4483    0.5778        29
           1     0.7576    0.9259    0.8333        27

    accuracy                         0.7595        79
   macro avg     0.7678    0.7769    0.7471        79
weighted avg     0.7707    0.7595    0.7386        79



  0%|          | 0/6 [00:00<?, ?it/s]